In [ ]:
#ALUNOS: JHONATAN MORAIS E PEDRO HENRIQUE

from collections import OrderedDict
from numpy import array, hsplit, concatenate, exp
import matplotlib.pyplot as plt


import pandas as pd
from pygenec.populacao import Populacao
from pygenec.selecao.roleta import Roleta
from pygenec.cruzamento.kpontos import KPontos
from pygenec.mutacao.flip import Flip
from pygenec.evolucao import Evolucao
from pygenec import AlgoritmoGenetico
from pygenec import binarray2int

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

class Mochila(AlgoritmoGenetico):
    def __init__(self, capacidade, pesos, maxob, cromossomos, selecao, cruzamento, mutacao, **kwargs):
        super(Mochila, self).__init__(
            selecao, cruzamento, mutacao, 1, cromossomos, **kwargs)
        self._pesos = pesos
        self._capacidade = capacidade
        self.maxob = maxob

    def lista_objetos(self, individuo):
        total = 0
        valor_total = 0
        objetos = OrderedDict()
        n_objs = 0
        npesos = len(self._pesos)

        for i in range(0, self._cromossomos, npesos):
            for k in range(len(self._pesos)):
                if individuo[i:i+npesos][k] == 1:
                    if total < self._capacidade:
                        if total + self._pesos[k][0] <= self._capacidade:
                            if k not in objetos:
                                objetos[k] = [self._pesos[k]]
                                total += self._pesos[k][0]
                                valor_total += self._pesos[k][1]
                                n_objs += 1
                            else:
                                if len(objetos[k]) < self.maxob:
                                    objetos[k].append(self._pesos[k])
                                    total += self._pesos[k][0]
                                    valor_total += self._pesos[k][1]
                                    n_objs += 1
                        else:
                          break
                    else:
                      break

        penalidade =  exp(( len(objetos.keys()) / npesos ))    
        optimizar = valor_total * penalidade

        return optimizar, n_objs, total, valor_total, objetos

    def funcao_objetivo(self, individuo):

        return self.lista_objetos(individuo)[0]


#ORÇAMENTO DE R$250
bits = 1
capacidade = 250 #orçamento
pesos = [[130, 30], #Tamiflu = item 0
         [30, 25], #Nitazoxanida = item 1
         [40, 45], #Azitromicina = item 2 
         [12, 55], #Ivermectina = item 3 
         [60, 50], #Cloroquina = item 4 
         [70, 50], #hidroxicloroquina = item 5 
         [100, 70], #Vitamina C = item 6
         [50, 65]] #Vitamina D = item 7 
cromossomos = 50 * len(pesos)
maxob = 1

algen = Mochila(capacidade, pesos, maxob, cromossomos, Roleta, KPontos, Flip, total_geracoes=100)
 
m = [_ for _ in algen()]

print(algen.lista_objetos(m[-1]))

df = pd.DataFrame([algen.lista_objetos(mi)[1:4] for mi in m], columns=['objetos', 'Preço', 'Eficácia'])

df2 = pd.DataFrame([[k, len(v)] for k, v in algen.lista_objetos(m[-1])[4].items()], columns=['objeto', 'contagem'])

df2.plot.bar(x='objeto',y='contagem')

df.plot(y='Eficácia')

df.plot(y='objetos')

algen2 = Mochila(capacidade, pesos, maxob, cromossomos, Roleta, KPontos, Flip,total_geracoes=500)
 
m2 = [_ for _ in algen2()]

print(algen2.lista_objetos(m2[-1]))

df = pd.DataFrame([algen2.lista_objetos(mi)[1:4] for mi in m2], columns=['objetos', 'Preço', 'Eficácia'])

df2 = pd.DataFrame([[k, len(v)] for k, v in algen2.lista_objetos(m2[-1])[4].items()], columns=['objeto', 'contagem'])

df2.plot.bar(x='objeto',y='contagem')

df.plot(y='Preço')

df.plot(y='Eficácia')

df.plot(y='objetos')